In [1]:
import os
import glob

# Define the root directory and simulation folder template
root_dir = "/home/yunan/pyilc/output/noise_res/"
sim_folder_template = "sim*"

# Define the filename templates
fn_template1 = "CN__needletcoeff_covmap_freq*_freq*_scale*.fits"
fn_template2 = "CN__needletcoeffmap_freq*_scale*.fits"
fn_template3 = "CN_needletILCmap_scale*_component_CMB_includechannels*_noise_res.fits"

# List of filename templates to delete
templates_to_delete = [fn_template1, fn_template2, fn_template3]

def delete_files(root_dir, sim_folder_template, templates_to_delete):
    # Find all simulation folders in the root directory
    sim_folders = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d)) and d.startswith("sim")]

    for sim_folder in sim_folders:
        print(f"Processing simulation folder: {sim_folder}")

        for template in templates_to_delete:
            # Generate the pattern for glob
            pattern = os.path.join(sim_folder, template)
            
            # Find files matching the pattern
            files_to_delete = glob.glob(pattern)

            for file_path in files_to_delete:
                try:
                    os.unlink(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

if __name__ == "__main__":
    delete_files(root_dir, sim_folder_template, templates_to_delete)

Processing simulation folder: /home/yunan/pyilc/output/noise_res/sim0000
Deleted: /home/yunan/pyilc/output/noise_res/sim0000/CN__needletcoeff_covmap_freq0_freq0_scale0.fits
Deleted: /home/yunan/pyilc/output/noise_res/sim0000/CN__needletcoeff_covmap_freq0_freq1_scale0.fits
Deleted: /home/yunan/pyilc/output/noise_res/sim0000/CN__needletcoeff_covmap_freq0_freq2_scale0.fits
Deleted: /home/yunan/pyilc/output/noise_res/sim0000/CN__needletcoeff_covmap_freq0_freq3_scale0.fits
Deleted: /home/yunan/pyilc/output/noise_res/sim0000/CN__needletcoeff_covmap_freq0_freq4_scale0.fits
Deleted: /home/yunan/pyilc/output/noise_res/sim0000/CN__needletcoeff_covmap_freq0_freq5_scale0.fits
Deleted: /home/yunan/pyilc/output/noise_res/sim0000/CN__needletcoeff_covmap_freq0_freq6_scale0.fits
Deleted: /home/yunan/pyilc/output/noise_res/sim0000/CN__needletcoeff_covmap_freq0_freq7_scale0.fits
Deleted: /home/yunan/pyilc/output/noise_res/sim0000/CN__needletcoeff_covmap_freq1_freq1_scale0.fits
Deleted: /home/yunan/pyilc/

In [3]:
def apodize_mask(mask, delta_c):

    npix = len(mask)
    nside = hp.npix2nside(npix)

    ipix = np.arange(npix)
    vec = np.array(hp.pix2vec(nside, ipix))


    apodized_mask = 1.0*mask

    good_pix = np.where(mask == 1)[0]
    bad_pix = mask == 0

    vec_bad = vec[:, bad_pix]

    for tmp in good_pix:
        vec_good = vec[:, tmp]
        ctheta = vec_bad[0, :]*vec_good[0] + vec_bad[1, :]*vec_good[1] + vec_bad[2, :]*vec_good[2]
        ctheta[ctheta > 1] = 1.0
        ctheta[ctheta < -1] = -1.0
        theta = np.arccos(ctheta)
        delta_i = np.degrees(np.min(theta))
        if delta_i <= delta_c:
            apodized_mask[tmp] = -1.0 / (2*np.pi) * np.sin(2*np.pi*delta_i / delta_c) + delta_i / delta_c
           
    return apodized_mask